In [1]:
import sys
import os
import networkx as nx

import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(os.path.join(os.getcwd(), '../..'))
from ConuForecast.src.graph_utils import GraphEngine, DBconnector

In [3]:
conn = DBconnector('172.17.0.1', 5555, 'base-ina', 'postgres', 'postgres')

EVENT = 'sjj-070'
MODEL = f'model_{EVENT}'
PRECIP = 'precipitation_{EVENT}'
ET = '2017-01-01 10:15:00'
ET = '2019-10-11 00:10:00'
ATTRS = {
    'nodes': ['elevation', 'area', 'imperv', 'slope','rainfall_acc', 'depth_above_invert'],
    'edges': []
    # 'edges': ['flow_rate', 'flow_velocity']
    }

conu_basin = GraphEngine(model=MODEL, event=EVENT, precip=PRECIP, conn=conn)

In [10]:
conu_basin.build_digraph(ET, ATTRS, in_place=False)

KeyboardInterrupt: 

In [11]:
preds_list = [(i[0],i[1]) for i in nx.edge_dfs(conu_basin.digraphs[f'{conu_basin.event}_{ET}'], 'NODO1819', 'reverse')]
# if len(preds_list) == 0:
#     preds_list = ['NODO1819',]

conu_basin.nodal_linkage_dict.items()
[(i[0],i[1]) for i in nx.edge_dfs(conu_basin.digraphs[f'{conu_basin.event}_{ET}'], 'NODO6987', 'reverse')][0]

('NODO7813', 'NODO6987')

In [9]:
conu_basin.build_subgraph('NODO6', ET, ATTRS, False, False)

In [10]:
','.join(['node_id', 'subcatchment_id', 'elapsed_time', 'depth_above_invert']) + ',' + ','.join(ATTRS['nodes'])

'node_id,subcatchment_id,elapsed_time,depth_above_invert,elevation,area,imperv,slope'

In [ ]:
'area', 'imperv', 'infiltration_loss', 'elevation', 


'flow_rate', 'length', 'roughness', 

In [5]:
conu_basin.nodal_data_dict['007_2017-01-01 00:15:00']['NODO7677']

{'node_id': 'NODO7677',
 'subcatchment_id': 'CUENCA7560',
 'elapsed_time': datetime.datetime(2017, 1, 1, 0, 15),
 'depth_above_invert': 0.0,
 'area': 0.278}

In [3]:
conu_basin.subgraph_tseries_viz('NODO1601',['area'], time_step=8)

TypeError: unhashable type: 'list'

In [6]:
a = conu_basin.build_subgraph('NODO1601', 'depth_above_invert'])

TypeError: build_subgraph() missing 2 required positional arguments: 'attrs' and 'acc_data'

In [5]:
conu_basin.timeseries_viz('NODO1601', ['depth_above_invert'])

AttributeError: 'list' object has no attribute 'startswith'

In [4]:
conu_basin.timeseries('NODO456', ['target'])

ValueError: Column `target` does not exist in the Frame

In [7]:
conu_basin.select_attrs({'nodes': ['total_inflow', 'lateral_inflow'], 'edges': ['flow_rate', 'flow_velocity']}, '2017-01-01 01:15:00')
graph_test = conu_basin.pruned_digraph
# graph_test.nodes(data=True)
# data = from_networkx(conu_basin.pruned_digraph)

In [8]:
to_torch(graph_test)

NameError: name &#39;torch&#39; is not defined

In [278]:
def to_torch(G):
    data = {}

    data['nodes_attrs'] = [list(v[1].values()) for i,v in enumerate(graph_test.nodes(data=True))]

    data['edge_attrs'] = [list(v[2].values()) for i,v in enumerate(graph_test.edges(data=True))]

    for key, item in data.items():
        try:
            data[key] = torch.tensor(item)
        except ValueError:
            pass
    data = torch_geometric.data.Data.from_dict(data)

    return data


In [282]:
to_torch(graph_test)['edge_index']

tensor([[   0,    1,    1,  ..., 8733, 8734, 8735],
        [   1, 5966, 5967,  ..., 8732, 8725, 8734]])

In [280]:
def to_torch(G):
    """Converts a :obj:`networkx.Graph` or :obj:`networkx.DiGraph` to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        G (networkx.Graph or networkx.DiGraph): A networkx graph.
    """

    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G
    edge_index = torch.tensor(list(G.edges)).t().contiguous()

    data = {}

    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        for key, value in feat_dict.items():
            data['node_' + str(key)] = [value] if i == 0 else data['node_' + str(key)] + [value]

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        for key, value in feat_dict.items():
            data['edge_' + str(key)] = [value] if i == 0 else data['edge_' + str(key)] + [value]
    
    data['nodes_attrs'] = [list(v[1].values()) for i,v in enumerate(G.nodes(data=True))]

    data['edge_attrs'] = [list(v[2].values()) for i,v in enumerate(G.edges(data=True))]

    for key, item in data.items():
        try:
            data[key] = torch.tensor(item)
        except ValueError:
            pass

    data['edge_index'] = edge_index.view(2, -1)
    data = torch_geometric.data.Data.from_dict(data)
    data.num_nodes = G.number_of_nodes()

    return data

In [225]:
conu_basin.select_attrs({'nodes': ['total_inflow', 'lateral_inflow'], 'edges': ['flow_rate']}, '2017-01-01 01:15:00')
graph_test = conu_basin.pruned_digraph
# data = from_networkx(conu_basin.pruned_digraph)

In [226]:
graph_test.nodes()['NODO7659']

{'lateral_inflow': 0.0, 'total_inflow': 5.9406293217989514e-09}

In [227]:
graph_test_torch = from_networkx(graph_test)
[type(i[1]) for i in graph_test_torch.items()]

[list, list, list, list, list, list]

In [228]:
graph_test_torch = from_networkx(graph_test)

graph_test_torch.zip

AttributeError: 'dict' object has no attribute 'zip'

In [201]:
list(zip(graph_test_torch.node_lateral_inflow, graph_test_torch.node_total_inflow))

8),
 (0.0, 1.6577952899332615e-10),
 (0.0, 2.4741353854551562e-08),
 (0.0, 3.203623200054295e-10),
 (0.0, 0.0),
 (0.0, 9.482099549984468e-09),
 (0.0, 1.3491977313151438e-07),
 (0.0, 6.154960097148887e-09),
 (0.0, 0.0),
 (0.0, 6.169269095579466e-09),
 (0.0, 6.921056172615181e-09),
 (0.0, 5.068490960269401e-09),
 (0.0, 3.322597308397235e-08),
 (0.0, 2.6533546915175066e-08),
 (0.0, 8.786339833477541e-08),
 (0.0, 0.0),
 (0.0, 3.970013651155568e-09),
 (0.0, 1.415124728509909e-07),
 (0.0, 0.0),
 (0.0, 3.5436546141909275e-08),
 (0.0, 3.8331140927994056e-08),
 (0.0, 3.930500369619949e-08),
 (0.0, 3.1279082435275996e-08),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 2.0391295763033668e-08),
 (0.0, 1.9692722119657446e-08),
 (0.0, 8.801228545962658e-08),
 (0.0, 1.1023355455108685e-06),
 (0.0, 8.743705315339412e-09),
 (0.0, 3.998681918915281e-08),
 (0.0, 2.3693151884884855e-09),
 (0.0, 0.0),
 (0.0, 1.7935006813019072e-09),
 (0.0, 2.5011811288777608e-08),
 (0.0, 5.996918162054499e-08),
 (0.0, 1.534851179485485

In [151]:
torch.tensor(list(zip(graph_test_torch.flow_rate, graph_test_torch.lateral_inflow, graph_test_torch.total_inflow)))

tensor([[4.7705e-08, 0.0000e+00, 3.8718e-08],
        [1.0125e-08, 0.0000e+00, 6.0930e-08],
        [1.8019e-08, 0.0000e+00, 6.1819e-08],
        ...,
        [2.4822e-10, 0.0000e+00, 0.0000e+00],
        [2.1795e-10, 0.0000e+00, 0.0000e+00],
        [9.6281e-09, 0.0000e+00, 0.0000e+00]])

In [ ]:
torch.nn(np.matrix(pd.DataFrame(graph.nodal_data_dict).T[['lateral_inflow', 'rainfall']]))

In [ ]:
nx.attr_matrix(a, node_attr='depth_above_invert')[0]

In [ ]:
torch.tensor([i for i in graph.nodal_data_query_result])